## 206a_FINAL_1_ShinahAudrey
Hi! This is Shinah. In this notebook, I will clean and organize all the dataset that will be used in the following notebooks and export them as geojson files. Operations I do in this notebook include dropping unnecessary columns, joining two files, changing data tytes to numeric, normalizing data by dividing data by total number or area and exporting each file to geojson.   

Dataset used:    
•LA County : population census data, jobs OnTheMap data, commute mode census data, rail ridership data.    
•Seoul :  population census data, number of workers(jobs) census data, commute mode census data, and rail ridership data.

### Cleaning dataset for LA

Import libraries

In [1]:
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Import population data

In [2]:
LA_pop = gpd.read_file('data/acs2019_5yr_B03002_14000US06037534001.geojson')

Dropping the first row because it skews the whole dataset.

In [3]:
LA_pop = LA_pop.drop([0])

Make a list of columns to keep

In [4]:
LA_pop_columnstokeep = ['geoid', 'name', 'B03002001', 'geometry']

Drop columns

In [5]:
LA_pop = LA_pop[LA_pop_columnstokeep]

Rename columns

In [6]:
LA_pop.columns = ['geoid', 'name', 'population', 'geometry']

For some reason this dataset is in France CRS so reprojecting: 

In [7]:
LA_pop = LA_pop.to_crs(epsg=3857)

Calculating LA population Density

In [8]:
#First calculate area in new column
LA_pop['area'] = LA_pop['geometry'].area

In [9]:
#Then calculate population density in new column
LA_pop['pop_density'] = LA_pop['population']/LA_pop['area']

In [10]:
LA_pop.head()

,geoid,name,population,geometry,area,pop_density
1,14000US06037101110,"Census Tract 1011.10, Los Angeles, CA",4283.0,"MULTIPOLYGON (((-13169350.794 4063591.834, -13...",1.676518e+06,0.002555
2,14000US06037101122,"Census Tract 1011.22, Los Angeles, CA",3405.0,"MULTIPOLYGON (((-13169467.902 4065613.937, -13...",3.880999e+06,0.000877
3,14000US06037101210,"Census Tract 1012.10, Los Angeles, CA",6347.0,"MULTIPOLYGON (((-13169034.646 4063225.625, -13...",9.547673e+05,0.006648
4,14000US06037101220,"Census Tract 1012.20, Los Angeles, CA",3702.0,"MULTIPOLYGON (((-13167528.939 4062726.771, -13...",1.025452e+06,0.003610
5,14000US06037101300,"Census Tract 1013, Los Angeles, CA",3884.0,"MULTIPOLYGON (((-13166671.668 4062511.964, -13...",3.787079e+06,0.001026


Import job data

In [11]:
LA_jobs = gpd.read_file('data/points_2018.shp')

In [12]:
LA_jobs.shape

(63568, 43)

In [13]:
LA_jobs.head()

,id,c000,ca01,ca02,ca03,ce01,ce02,ce03,cns01,cns02,...,cr07,ct01,ct02,cd01,cd02,cd03,cd04,cg01,cg02,geometry
0,060374600003006,12.0,3.0,3.0,6.0,0.0,8.0,4.0,0.0,0.0,...,0.0,10.0,2.0,0.0,4.0,3.0,2.0,7.0,5.0,POINT (-2001134.000 -138945.141)
1,060374300025013,3.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,POINT (-1991025.125 -145032.016)
2,060374300025015,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,POINT (-1991167.125 -145140.297)
3,060374300025014,21.0,10.0,8.0,3.0,9.0,12.0,0.0,0.0,0.0,...,0.0,10.0,11.0,1.0,2.0,2.0,6.0,12.0,9.0,POINT (-1991184.625 -145096.594)
4,060374300025021,4.0,0.0,3.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,2.0,2.0,0.0,3.0,1.0,POINT (-1991041.750 -145197.812)


Drop unnecessary columns 

In [14]:
keep_job_columns = ['id',
 'c000',
 'geometry']

In [15]:
LA_jobs = LA_jobs[keep_job_columns]

Rename columns

In [16]:
LA_jobs.columns = ['GEOID', 'Job Count', 'geometry']

In [17]:
LA_jobs = LA_jobs.to_crs(epsg=3857)

It is good to have the population and jobs layers in an identical coordinate reference systems. Convert jobs later to Web Mercator: 

In [18]:
LA_jobs.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Import commute data

In [19]:
LA_com = gpd.read_file('data/acs2019_5yr_B08006_14000US06037534001.geojson')

Dropping the first row to avoid skew

In [20]:
LA_com = LA_com.drop([0])

I checked the metadata and selected some columns that I want to keep.   
•'geoid':geoid of the census tract   
•'name':name of the census tract   
•'B08006001' : Total   
•'B08006002' : Car, truck, or van   
•'B08006008' : Public transportation (excluding taxicab)   
•'geometry' : long lat coordinates   

In [21]:
LA_com_columnstokeep = ['geoid', 'name', 'B08006001', 'B08006002', 'B08006008', 'geometry']

In [22]:
LA_com = LA_com[LA_com_columnstokeep]

Renaming the columns

In [23]:
LA_com.columns = ['geoid', 'Name', 'Total', 'Car, truck, or van', 'Public transportation (excluding taxicab)','geometry']

The raw data itself is less meaningful than looking at the percent share within each census tracts. So I divide the columns by the total number of commute trips.`

In [24]:
LA_com['Percent Car, truck, or van'] = LA_com['Car, truck, or van']/LA_com['Total']*100
LA_com['Percent Public transportation (excluding taxicab)'] = LA_com['Public transportation (excluding taxicab)']/LA_com['Total']*100

In [25]:
LA_com.head()

,geoid,Name,Total,"Car, truck, or van",Public transportation (excluding taxicab),geometry,"Percent Car, truck, or van",Percent Public transportation (excluding taxicab)
1,14000US06037101110,"Census Tract 1011.10, Los Angeles, CA",2027.0,1860.0,59.0,"MULTIPOLYGON (((-118.30229 34.25870, -118.3009...",91.761223,2.910705
2,14000US06037101122,"Census Tract 1011.22, Los Angeles, CA",1834.0,1730.0,9.0,"MULTIPOLYGON (((-118.30334 34.27371, -118.3033...",94.329335,0.490731
3,14000US06037101210,"Census Tract 1012.10, Los Angeles, CA",3057.0,2699.0,68.0,"MULTIPOLYGON (((-118.29945 34.25598, -118.2979...",88.289172,2.224403
4,14000US06037101220,"Census Tract 1012.20, Los Angeles, CA",1593.0,1439.0,48.0,"MULTIPOLYGON (((-118.28593 34.25227, -118.2859...",90.332706,3.013183
5,14000US06037101300,"Census Tract 1013, Los Angeles, CA",1782.0,1538.0,26.0,"MULTIPOLYGON (((-118.27822 34.25068, -118.2782...",86.307520,1.459035


Import rail ridership data

In [26]:
LA_ride = gpd.read_file('data/zRailLineDirHrStn_FY18_19.csv')

Cheking datatype to make sure the values are written in float

In [27]:
LA_ride.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 37067 entries, 0 to 37066
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   line                37067 non-null  object  
 1   route               37067 non-null  object  
 2   SER                 37067 non-null  object  
 3   Dir                 37067 non-null  object  
 4   stop_id             37067 non-null  object  
 5   StationName         37067 non-null  object  
 6   tripstarthour       37067 non-null  object  
 7   Daily_Boardings     37067 non-null  object  
 8   Daily_Alightings    37067 non-null  object  
 9   Average_Car_Load    37067 non-null  object  
 10  Average_Train_Load  37067 non-null  object  
 11  FY                  37067 non-null  object  
 12  geometry            0 non-null      geometry
dtypes: geometry(1), object(12)
memory usage: 3.7+ MB


I need to change datatype of columns I am interested in.

In [28]:
LA_ride["Daily_Boardings"] = pd.to_numeric(LA_ride["Daily_Boardings"], downcast="float")
LA_ride["Daily_Alightings"] = pd.to_numeric(LA_ride["Daily_Alightings"], downcast="float")
LA_ride["Average_Car_Load"] = pd.to_numeric(LA_ride["Average_Car_Load"], downcast="float")
LA_ride["Average_Train_Load"] = pd.to_numeric(LA_ride["Average_Train_Load"], downcast="float")

LA_ride.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 37067 entries, 0 to 37066
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   line                37067 non-null  object  
 1   route               37067 non-null  object  
 2   SER                 37067 non-null  object  
 3   Dir                 37067 non-null  object  
 4   stop_id             37067 non-null  object  
 5   StationName         37067 non-null  object  
 6   tripstarthour       37067 non-null  object  
 7   Daily_Boardings     27848 non-null  float32 
 8   Daily_Alightings    27848 non-null  float32 
 9   Average_Car_Load    27848 non-null  float32 
 10  Average_Train_Load  27848 non-null  float32 
 11  FY                  37067 non-null  object  
 12  geometry            0 non-null      geometry
dtypes: float32(4), geometry(1), object(8)
memory usage: 3.1+ MB


In [29]:
LA_ride.sort_values(by=['Daily_Boardings'], ascending = False)

,line,route,SER,Dir,stop_id,StationName,tripstarthour,Daily_Boardings,Daily_Alightings,Average_Car_Load,Average_Train_Load,FY,geometry
33627,802,802,DX,W,80214,Union Station - Metro Red & Purple Lines,7,2122.264648,0.0,59.0,355.0,FY19,None
29906,802,802,DX,W,80214,Union Station - Metro Red & Purple Lines,8,2112.641602,0.0,56.0,335.0,FY18,None
33628,802,802,DX,W,80214,Union Station - Metro Red & Purple Lines,8,2109.553223,0.0,56.0,338.0,FY19,None
29905,802,802,DX,W,80214,Union Station - Metro Red & Purple Lines,7,2106.920654,0.0,59.0,355.0,FY18,None
20516,801,801,DX,S,80122,7th Street / Metro Center Station - Metro Blue...,17,2004.324341,0.0,65.0,196.0,FY18,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29623,,,,,,,,NaN,NaN,NaN,NaN,,None
29624,,,,,,,,NaN,NaN,NaN,NaN,,None
29625,,,,,,,,NaN,NaN,NaN,NaN,,None
29626,,,,,,,,NaN,NaN,NaN,NaN,,None


In [30]:
 LA_ride.sort_values(by=['Daily_Alightings'], ascending = False)

,line,route,SER,Dir,stop_id,StationName,tripstarthour,Daily_Boardings,Daily_Alightings,Average_Car_Load,Average_Train_Load,FY,geometry
4050,802,802,DX,E,80214,Union Station - Metro Red & Purple Lines,17,0.0,2404.643555,0.0,0.0,FY19,None
300,802,802,DX,E,80214,Union Station - Metro Red & Purple Lines,17,0.0,2358.476807,0.0,0.0,FY18,None
4049,802,802,DX,E,80214,Union Station - Metro Red & Purple Lines,16,0.0,2132.331543,0.0,0.0,FY19,None
299,802,802,DX,E,80214,Union Station - Metro Red & Purple Lines,16,0.0,2028.575317,0.0,0.0,FY18,None
33364,802,802,DX,W,80201,North Hollywood Station,17,0.0,1825.754761,0.0,0.0,FY19,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29623,,,,,,,,NaN,NaN,NaN,NaN,,None
29624,,,,,,,,NaN,NaN,NaN,NaN,,None
29625,,,,,,,,NaN,NaN,NaN,NaN,,None
29626,,,,,,,,NaN,NaN,NaN,NaN,,None


(We received this dataset from LaMetro at almost the last momment of our project. We are already conducting analysis assuming that DTLA has the highest ridership, which is confirmed in the above cell. Thus, we stop cleaning this data here.)

Export dataframes to geojson to enable import in the following notebooks.

In [31]:
LA_pop.to_file("data/LA_pop.json", driver="GeoJSON")
LA_jobs.to_file("data/LA_jobs.json", driver="GeoJSON")
LA_com.to_file("data/LA_com.json", driver="GeoJSON")

### Cleaning dataset for Seoul

'seoul_data_com.csv' has population, number of workers, and commute data in it. It is the master spreadsheet for Seoul.

Import data

In [32]:
Seoul_data = gpd.read_file('data/seoul_data_com.csv')

Drop columns

In [33]:
Seoul_data_columns_to_keep = ['Gu', 'Dong',  'Population',  'Number of workers', 'Public_transportation', 'Car']

Redefine data

In [34]:
Seoul_data = Seoul_data[Seoul_data_columns_to_keep]

Import boundary shapefile (the above csv has no geometry)

In [35]:
adm_dong_seoul = gpd.read_file('data/adm_dong_seoul.shp')

Make a column that is a key for joining two datasets

In [36]:
adm_dong_seoul['Dong'] = adm_dong_seoul['ADM_DR_NM']

Merge and redefine data

In [37]:
adm_dong_seoul = adm_dong_seoul.merge(Seoul_data, on='Dong')

Change data type from object to string

In [38]:
adm_dong_seoul['Population'] = pd.to_numeric(Seoul_data['Population'])
adm_dong_seoul['Number of workers'] = pd.to_numeric(Seoul_data['Number of workers'])
adm_dong_seoul['Public_transportation'] = pd.to_numeric(Seoul_data['Public_transportation'])
adm_dong_seoul['Car'] = pd.to_numeric(Seoul_data['Car'])

Calculate population density and job density

In [39]:
adm_dong_seoul["area"]=adm_dong_seoul['geometry'].area

In [40]:
adm_dong_seoul["Pop_Density"]=adm_dong_seoul['Population']/adm_dong_seoul['area']
adm_dong_seoul["Work_Density"]=adm_dong_seoul['Number of workers']/adm_dong_seoul['area']

In [41]:
adm_dong_seoul.head()

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,OBJECTID,geometry,Dong,Gu,Population,Number of workers,Public_transportation,Car,area,Pop_Density,Work_Density
0,20200630,1101053,사직동,1,"POLYGON ((197702.069 453187.312, 197703.481 45...",사직동,종로구,4765,49536.0,57.0,15.3,1.158538e+06,0.004113,0.042757
1,20200630,1101054,삼청동,2,"POLYGON ((197980.839 455346.068, 197995.471 45...",삼청동,종로구,1369,4577.0,57.0,15.3,1.479250e+06,0.000925,0.003094
2,20200630,1101055,부암동,3,"POLYGON ((196621.073 456395.880, 196628.373 45...",부암동,종로구,4348,3609.0,57.0,15.3,2.274721e+06,0.001911,0.001587
3,20200630,1101056,평창동,4,"POLYGON ((197800.769 459064.245, 197782.531 45...",평창동,종로구,7518,3565.0,57.0,15.3,8.947324e+06,0.000840,0.000398
4,20200630,1101057,무악동,5,"POLYGON ((196444.745 453384.564, 196471.668 45...",무악동,종로구,3203,1598.0,57.0,15.3,3.695239e+05,0.008668,0.004324


Import Seoul railway point data    
(In Seoul Metropolitan area, there are about 20 lines including lines that are operated by SeoulMetro and some private-owned lines. Our shapefile includes the 10 main lines that are mainly used within Seoul and the outskirts.)

In [42]:
Seoul_rail_point = gpd.read_file('data/seoul_subway_ridership_joined.shp')

Import rail ridership data

In [43]:
Seoul_rail_ridership = gpd.read_file('data/seoul_subway_ridership_december.csv')

These two data will be joined. 

Make a list of columns to keep

In [44]:
rail_columnstokeep = ['line', 'st_nm', 'ridership']

In [45]:
Seoul_rail_ridership = Seoul_rail_ridership[rail_columnstokeep]

Create column that will work as key to join

In [46]:
Seoul_rail_point['st_nm'] = Seoul_rail_point['stn_name']

Join a shapefile and csv

In [47]:
Seoul_rail_point = Seoul_rail_point.merge(Seoul_rail_ridership, on='st_nm')

In [48]:
Seoul_rail_point.crs

<Projected CRS: EPSG:5174>
Name: Korean 1985 / Modified Central Belt
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Korea, Republic of (South Korea) - 126°E to 128°E mainland
- bounds: (126.0, 33.96, 128.0, 38.33)
Coordinate Operation:
- name: Korea Modified Central Belt
- method: Transverse Mercator
Datum: Korean Datum 1985
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [49]:
Seoul_rail_point.head()

,번호,stn_name,adm(leg)_d,stn_addres,zipcode,입력주소,X,Y,CLSS,PNU,...,표준신주소,표준구주소,우편번호,seoul_subw,seoul_su_1,seoul_su_2,geometry,st_nm,line,ridership
0,2,시청(1)역,명동(태평로1가),서울시 중구 태평로1가 31,4519,서울시 중구 태평로1가 31,198018,451601,정좌표,공개용버전 미지원,...,미지원,미지원,04524,1호선,시청(1)역,852766,POINT (198018.000 451601.000),시청(1)역,1호선,852766
1,3,종각(1)역,종로1.2.3.4가동(종로1가),서울시 종로구 종로1가 44,3161,서울시 종로구 종로1가 44,198343,452018,인근좌표,공개용버전 미지원,...,미지원,미지원,NULL,1호선,종각(1)역,1309829,POINT (198343.000 452018.000),종각(1)역,1호선,1309829
2,6,동대문(1)역,창신제1동(창신동),서울시 종로구 창신1동 552,3119,서울시 종로구 창신1동 552,201032,452179,인근좌표,공개용버전 미지원,...,미지원,미지원,NULL,1호선,동대문(1)역,504353,POINT (201032.000 452179.000),동대문(1)역,1호선,504353
3,7,동묘앞(1)역,숭인제1동(숭인동),서울시 종로구 숭인동 117,3113,서울시 종로구 숭인동 117,201444,452367,정좌표,공개용버전 미지원,...,미지원,미지원,03113,1호선,동묘앞(1)역,423253,POINT (201444.000 452367.000),동묘앞(1)역,1호선,423253
4,8,신설동(1)역,용신동(신설동),서울시 동대문구 신설동 97-75,2582,서울시 동대문구 신설동 97-75,202246,452552,인근좌표,공개용버전 미지원,...,미지원,미지원,NULL,1호선,신설동(1)역,613813,POINT (202246.000 452552.000),신설동(1)역,1호선,613813


Make a list of columns to keep. Joined file above has unnecessary columns.

In [50]:
Seoul_rail_columnstokeep = ['line','stn_name','st_nm', 'ridership', 'geometry']

Redefine the file.

In [51]:
Seoul_rail_point = Seoul_rail_point[Seoul_rail_columnstokeep]

Change data type to string

In [52]:
Seoul_rail_point['ridership'] = pd.to_numeric(Seoul_rail_point['ridership'])

In [53]:
Seoul_rail_point.head()

,line,stn_name,st_nm,ridership,geometry
0,1호선,시청(1)역,시청(1)역,852766,POINT (198018.000 451601.000)
1,1호선,종각(1)역,종각(1)역,1309829,POINT (198343.000 452018.000)
2,1호선,동대문(1)역,동대문(1)역,504353,POINT (201032.000 452179.000)
3,1호선,동묘앞(1)역,동묘앞(1)역,423253,POINT (201444.000 452367.000)
4,1호선,신설동(1)역,신설동(1)역,613813,POINT (202246.000 452552.000)


Export files as geojson

In [ ]:
adm_dong_seoul.to_file("data/Seoul_master.json", driver="GeoJSON")
Seoul_rail_point.to_file("data/Seoul_rail_ridership.json", driver="GeoJSON")

This is the end of the notebook1. In the next notebook, '206a_FINAL_2_ShinahAudrey', I will conduct citywide analysis and plot histograms and maps.